# Chapter 3: GPU programming and machine learning libraries.

Structure:
- [CuPy](#CuPy)
- [Numba](#Numba)
- [CuDF](#CuDF)
- [CuML](#CuML)
- [Pytorch](#Pytorch)
- [Tensorflow and Keras](#TF)
- [TODO](#TODO)

<a name="CuPy"></a>
## CuPy

<a name="Numba"></a>
## Numba

<a name="CuDF"></a>
## CuDF

<a name="CuML"></a>
## CuML

<a name="Pytorch"></a>
## Pytorch

<a name="TF"></a>
## Tensorflow and Keras

<a name="TODO"></a>
## TODO